Frequently Bought Toghether Items

Use only if on Colab and there is a _COLAB folder in your Google Drive with necessary files

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')
import os
os.chdir("drive/MyDrive/_COLAB")

Initialize graphs etc

In [24]:
import random
import networkx as nx
import node2vec
from gensim.models import Word2Vec

#Nodo target
target = 3

#Grafo completo FBT
full_graph = nx.Graph()

#Grafo cluster selezionato
clustered_graph = nx.Graph()

#File di lettura
file_name = "Amazon0302.txt"
#file_name = "grafo_esempio.txt"

#Grafo directed se True o uniderected se False
directed = True

#embedding hyperparameters
#default = 1, 1, 10, 80
p=1
q=1
num_walks=10
walk_length=80

In [ ]:
#Author:Savoia Emanuele
### LOAD GRAPH ###
print("loading graph " + file_name)

if directed:
    ##genera il grafo directed utilizzando nx.DiGraph
    full_graph = nx.read_edgelist(file_name, nodetype=int, create_using=nx.DiGraph)

else:
    ##genera il grafo undirected utilizzando nx.Graph
    full_graph = nx.read_edgelist(file_name)

nx.set_edge_attributes(full_graph, 1, name='weight')
print(full_graph)

print("finished")

Embedding using stanford's node2vec

In [ ]:
#Author:Savoia Emanuele
G = node2vec.Graph(full_graph, directed, p, q)
G.preprocess_transition_probs()
walks = G.simulate_walks(num_walks, walk_length)

In [ ]:
#Author:Savoia Emanuele
walks = [list(map(str, walk)) for walk in walks]
model = Word2Vec(walks, window=10, min_count=0, sg=1, workers=100)
#model.save("out.model") ##uncomment to save vectors

Kmeans and sampled kmeans implementations

In [ ]:
#Author:Savoia Emanuele
print(model)
vector = model.wv[1]
print(vector)

import threading
sem = threading.Semaphore()
sem_print = threading.Semaphore()

NUMBER_OF_ITERATIONS = 10

#k_means_cluster
class Point:
    def __init__(self, x, y):
        self.vector = x
        self.assigned = y

#calculate distance between two points
def distance(point1, point2):
    dist = [(a - b)**2 for a, b in zip(point1.vector, point2.vector)]
    return math.sqrt(sum(dist))

#calculate squared distance between two points
def distance_sq(point1, point2):
    dist = [(a - b)**2 for a, b in zip(point1.vector, point2.vector)]
    return sum(dist)

#calculate S score of point given array of centers and other points
def calculate_silhouette(point, centers, points):
    a_i = 0.0
    b_i = float('inf')
    cluster_size = 0
    members = 0

    for i in range(len(points)):
        if points[i].assigned == point.assigned:
            a_i += distance(point, points[i])
            members += 1

    if members < 2:
        return 0
    else:
        a_i/= (members-1)

    for k in range(len(centers)):
        if not(k == point.assigned):
            sum = 0.0
            number = 0
            for i in range(len(points)):
                if points[i].assigned == k:
                    sum += distance(point, points[i])
                    number +=1
            if not(number == 0):
                sum /= number
                if sum < b_i:
                    b_i = sum

    if len(centers) == 1:
        b_i = 0

    return (b_i - a_i)/max(a_i, b_i)

# calculate total silhouette score for clustering
def calculate_silhouette_score(points, centers):
    ss = 0.0
    for point in points:
        best_assign = -1
        min_dist = float('inf')
        for k in range(len(centers)):
            if distance(point, centers[k]) < min_dist:
                point.assigned = k
                min_dist = distance(point, centers[k])

    for p in points:
        ss += calculate_silhouette(p, centers, points)

    return ss/len(points)

def calculate_silhouette_score_sampled(points, centers, sample_size):
    ss = 0.0
    pts_sampled = random.sample(points, sample_size)
    for point in pts_sampled:
        best_assign = -1
        min_dist = float('inf')
        for k in range(len(centers)):
            if distance(point, centers[k]) < min_dist:
                point.assigned = k
                min_dist = distance(point, centers[k])

    for p in pts_sampled:
        ss += calculate_silhouette(p, centers, pts_sampled)

    return ss/len(pts_sampled)

def k_means_plus_plus_init(points, k_chosen):
    ##init with kmeans++##
    centroids = []
    distances = []
    probabilities = []
    centroids_index = []

    random_first_point = random.randrange(len(points))
    centroids.append(points[random_first_point])
    centroids_index.append(random_first_point)
    sum = 0.0
    for p in points:
        p.assigned = 0
        current = distance_sq(centroids[0], p)
        distances.append(current)
        sum += current
        probabilities.append(sum)
    #choose a point with weighted probability for kmeans ++
    for i in range(1, k_chosen):
        print("Finding centroid number " + str(i))
        chosen = random.uniform(0.0, sum)
        c = 0
        for j in range(len(points)):
            if chosen > probabilities[j]:
              continue
            else:
              if j == 0:
                c = 0
                break
              else:
                c = j - 1
                new = False
                while not new:
                  for K_old in centroids_index:
                    if probabilities[c] == probabilities[K_old]:
                      c -= 1
                      new = False
                      break
                    else:
                      new = True
                if c < 0:
                  c = j
                break
        centroids_index.append(c)
        centroids.append(points[c])
        sum = 0
        for j in range(len(points)):
            current = distance_sq(centroids[i], points[j])
            if(distances[j] > current):
                distances[j] = current
                points[j].assigned = i
            sum += distances[j]
            probabilities[j] = sum
    return centroids, centroids_index

def k_means_plus_plus(points, k_chosen, max_iter = NUMBER_OF_ITERATIONS):
    ##init with kmeans++##
    centroids, centroids_index = k_means_plus_plus_init(points, k_chosen)

    for iter in range(max_iter):
        print("Starting kmeans iter " + str(iter) + "/" +str(max_iter))
        changed = False
        for k in range(k_chosen):
            med_vec = []
            for j in range(len(points[0].vector)):
                med = 0
                pts = 0
                for p in points:
                    if(p.assigned == k):
                        med += p.vector[j]
                        pts += 1
                if not(pts == 0):
                  med /= pts
                else:
                  print("FOR K = " + str(k) + "zero points were found during iter: " + str(iter))
                med_vec.append(med)
            centroids[k] = Point(med_vec, k)

        for p in points:
            best_dist = float('inf')
            best_k = 0
            for k in range(k_chosen):
                #use square to save time
                dist = distance_sq(p, centroids[k])
                if(dist < best_dist):
                    best_dist = dist
                    best_k = k
            if not(best_k == p.assigned):
                changed = True
                p.assigned = best_k

        if not changed:
            break
        else:
            changed = False

    #find closest point to centroid
    for k in range(k_chosen):
        dist = float('inf')
        i = -1
        for p in range(len(points)):
          #square to save time
            dist_p = distance_sq(points[p], centroids[k])
            if dist_p < dist :
                dist = dist_p
                i = p
        centroids[k] = points[i]

    return centroids

def obj_fun(points, centroids):
  sum = 0.0
  for p in points:
    mindist = float('inf')
    for k in range(len(centroids)):
      dist = distance_sq(centroids[k], p)
      if dist < mindist:
        p.assigned = k
        mindist = dist
    sum += mindist
  return sum

def bigKmeansThread(points, k_chosen, sample_size, best_centroids, fopt, result, index):
  print("Starting thread number: " + str(index))
  pts_sampled = random.sample(points, sample_size)
  centroids = k_means_plus_plus(pts_sampled, k_chosen)
  this_f = obj_fun(pts_sampled, centroids)
  sem.acquire()
  if this_f < fopt[0]:
    best_centroids = centroids
    fopt[0] = this_f
    result[index] = True
  else:
    result[index] = False
  sem.release()

def bigKmeans(points, k_chosen, sample_size, num_threads = 16, stop_after = 10, max_iter = 3):
  best_centroids = []
  stop = 0
  fopt = [float('inf')] ##useful for threading purposes
  iter = 0
  while stop < stop_after and iter < max_iter:
    threads = [None] * num_threads
    results = [None] * num_threads

    for i in range(len(threads)):
      threads[i] = threading.Thread(target=bigKmeansThread, args=(points, k_chosen, sample_size, best_centroids, fopt, results, i))
      threads[i].start()

    for i in range(len(threads)):
      threads[i].join()

    for i in range(len(results)):
      if results[i]:
        stop += 1
    iter += 1
    print("############################################################")
    print("Better result number " + str(stop) + "/" + str(stop_after))
    print("Iteration number " + str(iter) + "/" + str(max_iter))
    print("Best score: " + str(fopt[0]))
    print("############################################################")
  return best_centroids

def find_best_centroids(points):
    best_centroids = []
    best_score = -1.0
    for k in range(int(len(points)/15), len(points)):
      print("trying K = " + str(k))
      current_score = -1
      current_centroids = k_means_plus_plus(points, k)
      current_score = calculate_silhouette_score(points, current_centroids)
      if(current_score >= best_score):
          best_centroids = current_centroids
          best_score = current_score
      else:
            return best_centroids

def find_best_centroids_big(points, sample_size):
    best_centroids = []
    best_score = -1.0
    for k in range(int(sample_size/15), sample_size):
      print("trying K = " + str(k))
      current_score = -1
      current_centroids = bigKmeans(points, k, sample_size)
      current_score = calculate_silhouette_score_sampled(points, current_centroids, sample_size)
      if(current_score >= best_score):
          best_centroids = current_centroids
          best_score = current_score
      else:
            return best_centroids

Run kmeans

In [ ]:
#Author:Savoia Emanuele
points_ = []
#initialize vector of points to be used in Kmeans++
for p in range(full_graph.number_of_nodes()):
    points_.append(Point(model.wv[p], 0))

Usage: uncomment the function you want to use.

Use Kmeans ++ and silhouette score computation to find a good clustering, VERY SLOW

    > best_c = find_best_centroids(points_)

Use Kmeans ++ computation to find a good clustering given k, VERY SLOW

    > best_c = k_means_plus_plus(points_, k)

Use Kmeans ++ and silhouette score computation on random samples to find a good clustering, SLOW

    > best_c = find_best_centroids_big(points_, sample_size)

Use Kmeans ++ computation to find a good clustering given k on random samples, FASTEST

    > best_c = bigKmeans(points_, k, sample_size)

In [ ]:
#Author:Savoia Emanuele
import math
best_c = []
sample_size = int(math.sqrt(len(points_)))
print("calculating best k-means using a sample of " + str(sample) + " vectors")
k = int(sample_size/20)
##best_c = find_best_centroids(points_)
##best_c = k_means_plus_plus(points_, k)
#best_c = bigKmeans(points_, k, sample_size)
best_c = find_best_centroids_big(points_, sample_size)
clustered = []

In [ ]:
#Author:Savoia Emanuele
for p in points_:
    min_dist = float('inf')
    for k in range(len(best_c)):
        dist_ = distance(p, best_c[k])
        if(dist_ < min_dist):
            min_dist = dist_
            p.assigned = k

target_cluster = points_[target].assigned

for p in range(len(points_)):
    if points_[p].assigned == target_cluster:
        clustered.append(p)

print(clustered)
clustered_graph = full_graph.subgraph(clustered)
print("Clustered graph: " + str(clustered_graph))
#Remove nodes that are not connected
if(directed):
  u = clustered_graph.to_undirected()
  nodes = nx.node_connected_component(u, target)
  clustered_graph = clustered_graph.subgraph(nodes)
else:
  clustered_graph = nx.node_connected_component(clustered_graph, target)
print("Clustered graph's connected component with target node: " + str(clustered_graph))

In [ ]:
#Clustering global cc
average_cluster_coef = nx.average_clustering(clustered_graph)

print(f"Coefficiente di clustering medio: {average_cluster_coef}")



#Clustering per ogni nodo (restituisce un dizionario)
node_cluster_coefs = nx.clustering(clustered_graph)

for node, cluster_coef in node_cluster_coefs.items():
    print(f"Nodo {node}: Coefficiente di clustering = {cluster_coef}")


Valutazione nodi (usare clustered_graph)

In [ ]:
#Valutazione nodi con CC o altre metriche

In [ ]:
#Ricerca cliques

#lettura file di esempio, costruzione nodes e edges
import os
import networkx as nx

current_directory = os.path.dirname(os.path.realpath("__file__"))
filename = "grafo_esempio.txt"

graph = nx.Graph()
graph = nx.read_edgelist(filename, nodetype=int, create_using=nx.DiGraph)
nx.set_edge_attributes(graph, 1, name='weight')
print(graph)

print("Edges:", graph.edges)
print("Unique Node IDs:", graph.nodes)

In [32]:
#funzione per il neighbor
def n(v, edges):
    neighbors = set()
    for edge in edges:
        if v in edge:
            neighbors.update(edge)

    neighbors.discard(v)

    return list(neighbors)

#Bron-Kerbosch algorithm with pivot
def BronKerbosch(R, P, X, edges, cliques):
    if not P and not X:
        # P and X are both empty, report R as a maximal clique
        cliques.append(R)
        return

    # Choose a pivot vertex u in P ⋃ X
    pivot = (set(P) | set(X)).pop()

    for v in set(P) - set(n(pivot, edges)):
        # Recursively explore the neighborhood of v
        BronKerbosch(R + [v], list(set(P) & set(n(v, edges))), list(set(X) & set(n(v, edges))), edges, cliques)

        # Remove v from P and add it to X
        P.remove(v)
        X.append(v)

#funzione per la clique massima
def maxClique(cliques):
    return max(cliques, key=len, default=[])

#funzione per clique del target
def nodeCliques(all_cliques, selected_node):
    node_cliques = []
    for clique in all_cliques:
        if selected_node in clique:
            node_cliques.append(clique)
    return node_cliques

In [33]:
#Clustering global cc
def runNodeCC(graph):
    #average_cluster_coef = nx.average_clustering(graph)
    #print(f"Coefficiente di clustering medio: {average_cluster_coef}")

    #Clustering per ogni nodo (restituisce un dizionario)
    node_cluster_coefs = nx.clustering(graph)

    #for node, cluster_coef in node_cluster_coefs.items():
        #print(f"Nodo {node}: Coefficiente di clustering = {cluster_coef}")
        
    return node_cluster_coefs


In [34]:
#node clique with max cc
def bestClique(node_cliques, node_cluster_coefs):
    max_cc = 0
    best_clique = []
    for clique in node_cliques:
        avg_cc = 0;
        for node in clique:
            avg_cc += node_cluster_coefs[node]

        if avg_cc/len(clique) > max_cc:
            best_clique = clique
    return best_clique

In [ ]:
#Ricerca e stampa le cliques e la clique massima
all_cliques = []
selected_node = 3
BronKerbosch([], list(graph.nodes), [], graph.edges, all_cliques)
node_cliques = nodeCliques(all_cliques, selected_node)
print("Cliques nodo:")
print(node_cliques)
print("Best clique")
print(bestClique(node_cliques, runNodeCC(graph)))

In [ ]:
#Tentativi con rimozioni casuali
import random

# Numero di nodi da rimuovere ad ogni iterazione
n_nodes_to_remove = 10

# Numero totale di iterazioni
n_iterations = 100

# Copia del grafo iniziale
graph_to_modify = graph.copy()

#results
results = []

for iteration in range(n_iterations):
    # Escludere il target
    filtered_nodes = [element for element in list(graph_to_modify.nodes()) if element != selected_node]
    
    # Rimuovi n nodi casuali dal grafo
    nodes_to_remove = random.sample(filtered_nodes, n_nodes_to_remove)
    updated_graph = graph_to_modify.copy()
    updated_graph.remove_nodes_from(nodes_to_remove)

    # Stampa o elabora i risultati desiderati
    print(f"Iteration {iteration + 1}: Removed nodes {nodes_to_remove}")

    all_cliques = []
    BronKerbosch([], list(updated_graph.nodes), [], updated_graph.edges, all_cliques)
    node_cliques = nodeCliques(all_cliques, selected_node)
    print("Cliques nodo:")
    print(node_cliques)
    best_clique = bestClique(node_cliques, runNodeCC(updated_graph))
    print("Best clique")
    print(best_clique)
    results.append(best_clique)
    
    # Aggiorna il grafo originale per la prossima iterazione
    #graph_to_modify = updated_graph.copy()


In [ ]:
#plot results

from collections import Counter
import matplotlib.pyplot as plt

def generate_histogram(array_of_arrays):
    # Convert inner arrays to tuples and use Counter
    element_counts = Counter(tuple(inner_array) for inner_array in array_of_arrays)

    # Extract elements and their counts
    elements = list(element_counts.keys())
    counts = list(element_counts.values())

    # Plot the histogram
    plt.bar(range(len(elements)), counts, align='center')
    plt.xticks(range(len(elements)), elements)
    plt.xlabel('Best clique')
    plt.ylabel('Count')
    plt.title('Histogram of results obtained')
    
    # Annotate each bar with its count
    for i, count in enumerate(counts):
        plt.text(i, count + 0.1, str(count), ha='center', va='bottom')
    
    plt.show()

generate_histogram(results)